<a href="https://colab.research.google.com/github/JerryLiu789/Stock-Price-Prediction/blob/main/Stock-Price-Prediction(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**月營收爬蟲**

In [ ]:
import requests
import pandas as pd
from io import StringIO

# 公開資訊觀測站 https://mops.twse.com.tw/mops/web/t21sc04_ifrs


url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_113_5_0.html'
r = requests.get(url)
r.encoding = 'big5'

# 將html轉換為檔案
dfs = pd.read_html(StringIO(r.text))
dfs

[                                                   0
 0  產業別：水泥工業單位：千元 營業收入累計營業收入備註公司 代號公司名稱當月營收上月營收去年當...,
                                             產業別：水泥工業  \
 0  營業收入累計營業收入備註公司 代號公司名稱當月營收上月營收去年當月營收上月比較 增減(%)去...   
 
                                                單位：千元  
 0  營業收入累計營業收入備註公司 代號公司名稱當月營收上月營收去年當月營收上月比較 增減(%)去...  ,
   Unnamed: 0_level_0 Unnamed: 1_level_0      營業收入                      \
                公司 代號               公司名稱      當月營收      上月營收    去年當月營收   
 0               1101                 台泥  13860276  11626493   9596091   
 1               1102                 亞泥   6459670   5686606   7238507   
 2               1103                 嘉泥    243833    238141    227797   
 3               1104                 環泥    743116    686687    671021   
 4               1108                 幸福    435713    426739    482963   
 5               1109                 信大    516800    441921    627486   
 6               1110                 東泥    204577    207095    213937

In [ ]:
# 將dfs中，row長度介於5~11的table合併，取出需要的 Table
df = pd.concat([df for df in dfs[2:] if df.shape[1] <= 11 and df.shape[1] > 5])
df

Unnamed: 0_level_0 Unnamed: 1_level_0        營業收入                          \
                公司 代號               公司名稱        當月營收        上月營收      去年當月營收   
0                1101                 台泥    13860276    11626493     9596091   
1                1102                 亞泥     6459670     5686606     7238507   
2                1103                 嘉泥      243833      238141      227797   
3                1104                 環泥      743116      686687      671021   
4                1108                 幸福      435713      426739      482963   
..                ...                ...         ...         ...         ...   
42               9944                 新麗      209139      223904      193827   
43               9945                潤泰新     2343898     2687510     2879621   
44                 合計                 合計    40018401    37401728    34924881   
0                  合計                 合計           0           0           0   
0          全部國內上市公司合計         全部國內上市公司合計  3305111909  3200945062  2846712942   

                               累計營業收入                           備註  
   上月比較 增減(%) 去年同月 增減(%)       當月累計營收       去年累計營收 前期比較 增減(%)   備註  
0       19.21      44.43     51172432     45057867      13.57    -  
1       13.59     -10.75     28646108     33616090     -14.78    -  
2        2.39       7.03      1181116      1099471       7.42    -  
3        8.21      10.74      3338375      3150457       5.96    -  
4        2.10      -9.78      2055420      1971812       4.24    -  
..        ...        ...          ...          ...        ...  ...  
42      -6.59       7.89      1169000      1010312      15.70    -  
43     -12.78     -18.60     12013791     11337369       5.96    -  
44       6.99      14.58    187136082    158141057      18.33  NaN  
0         NaN        NaN            0            0        NaN  NaN  
0        3.25      16.10  15437926528  14003520063      10.24  NaN  

[970 rows x 11 columns]

In [ ]:
# 處理多層列標籤
df.columns = df.columns.get_level_values(1)

# 設定column名稱
df = df.rename(columns={'公司 代號': '公司代號'})

# 將 df 中的當月營收用 .to_numeric 轉換為數值，再把其中不能轉換為數值的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')

# 刪除當月營收中出現 NaN 的 row
df = df[~df['當月營收'].isnull()]

# 刪除公司代號中包含 合計 的 row
df = df[df['公司代號'] != '合計']

# 將 公司代號 與 公司名稱 共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])

df

,,當月營收,上月營收,去年當月營收,上月比較 增減(%),去年同月 增減(%),當月累計營收,去年累計營收,前期比較 增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,13860276,11626493,9596091,19.21,44.43,51172432,45057867,13.57,-
1102,亞泥,6459670,5686606,7238507,13.59,-10.75,28646108,33616090,-14.78,-
1103,嘉泥,243833,238141,227797,2.39,7.03,1181116,1099471,7.42,-
1104,環泥,743116,686687,671021,8.21,10.74,3338375,3150457,5.96,-
1108,幸福,435713,426739,482963,2.10,-9.78,2055420,1971812,4.24,-
...,...,...,...,...,...,...,...,...,...,...
9941,裕融,3493194,3468896,3569498,0.70,-2.13,17356658,16662469,4.16,-
9942,茂順,355960,310472,431470,14.65,-17.50,1620340,1990194,-18.58,-
9944,新麗,209139,223904,193827,-6.59,7.89,1169000,1010312,15.70,-


In [ ]:
# 功能測試，存成CSV檔
df.to_csv('test.csv', encoding = 'utf_8_sig')

# 功能測試，讀取CSV檔
df = pd.read_csv('test.csv',index_col = ['公司代號', '公司名稱'])
df.head()

,,當月營收,上月營收,去年當月營收,上月比較 增減(%),去年同月 增減(%),當月累計營收,去年累計營收,前期比較 增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,13860276,11626493,9596091,19.21,44.43,51172432,45057867,13.57,-
1102,亞泥,6459670,5686606,7238507,13.59,-10.75,28646108,33616090,-14.78,-
1103,嘉泥,243833,238141,227797,2.39,7.03,1181116,1099471,7.42,-
1104,環泥,743116,686687,671021,8.21,10.74,3338375,3150457,5.96,-
1108,幸福,435713,426739,482963,2.10,-9.78,2055420,1971812,4.24,-


In [ ]:
# 功能測試，存到 sqlite3 中
import sqlite3
conn = sqlite3.connect('test.sqlite3')
df.to_sql('monthly_report', conn, if_exists = 'replace')

937

In [ ]:
# 功能測試，讀取 sqlite3 內容
df = pd.read_sql('select * from monthly_report', conn, index_col=['公司代號', '公司名稱'])
df.head()

,,當月營收,上月營收,去年當月營收,上月比較 增減(%),去年同月 增減(%),當月累計營收,去年累計營收,前期比較 增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,13860276,11626493,9596091,19.21,44.43,51172432,45057867,13.57,-
1102,亞泥,6459670,5686606,7238507,13.59,-10.75,28646108,33616090,-14.78,-
1103,嘉泥,243833,238141,227797,2.39,7.03,1181116,1099471,7.42,-
1104,環泥,743116,686687,671021,8.21,10.74,3338375,3150457,5.96,-
1108,幸福,435713,426739,482963,2.10,-9.78,2055420,1971812,4.24,-
